In [ ]:
dbutils.widgets.text("input", "","")
pipelineId = dbutils.widgets.get("input")
print ("Pipeline run id:")
print (pipelineId)

In [ ]:
accountName = dbutils.secrets.get("dataLakeScope","accountName")
accountKey = dbutils.secrets.get("dataLakeScope","accountKey")
sparkProperty = f'fs.azure.account.key.{accountName}.dfs.core.windows.net'
spark.conf.set(sparkProperty,accountKey)

In [ ]:
storagePath = f'abfss://bronze@{accountName}.dfs.core.windows.net/nybabynames.csv'
df = spark.read.option("inferSchema", "true").csv(path= storagePath, header=True)

display(df.printSchema)
display(df)